# ✈️ Travel Planner with APIs
This notebook allows you to:
- Get current weather of a city
- See 5 top attractions with clickable links
- Check the current exchange rate between CAD and local currency

In [1]:
from IPython.display import Image, display, Markdown
import requests
from datetime import datetime, timedelta
import sqlite3
from dotenv import load_dotenv
import os
load_dotenv()

weather_key = os.getenv("OPENWEATHER_API_KEY")
unsplash_key =  os.getenv("UNSPLASH_ACCESS_KEY")
google_key = os.getenv("GOOGLE_API_KEY")
exchange_key = os.getenv("EXCHANGE_API_KEY")

# === Function: Get Weather ===
def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_key}&units=metric"
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        offset = data["timezone"]
        local_time = (datetime.utcnow() + timedelta(seconds=offset)).strftime('%Y-%m-%d %H:%M:%S')
        return {
            "description": data["weather"][0]["description"],
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "local_time": local_time,
            "country": data["sys"]["country"],
            "lat": data["coord"]["lat"],
            "lon": data["coord"]["lon"]
        }
    else:
        return {"error": data.get("message", "Something went wrong.")}

# === Function: Get Currency Code ===
def get_currency_code(country_code):
    currency_map = {
        "US": "USD", "JP": "JPY", "FR": "EUR", "GB": "GBP",
        "CN": "CNY", "CA": "CAD", "AU": "AUD"
    }
    return currency_map.get(country_code.upper(), None)

# === Function: Get Exchange Rate ===
def get_exchange_rate(from_currency="CAD", to_currency="USD"):
    url = f"https://v6.exchangerate-api.com/v6/{exchange_key}/pair/{from_currency}/{to_currency}"
    res = requests.get(url).json()
    return res["conversion_rate"] if res["result"] == "success" else "Rate unavailable"

# === Function: Get Top Places (Hotels, Restaurants, Attractions) ===
def get_top_places(lat, lon, place_type):
    url = (
        f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        f"?location={lat},{lon}&radius=5000&type={place_type}&opennow=true&key={google_key}"
    )
    response = requests.get(url)
    data = response.json()
    results = []
    if data.get("status") == "OK":
        for place in data.get("results", [])[:5]:
            name = place.get("name", "Unknown")
            address = place.get("vicinity", "No address")
            place_id = place.get("place_id", "")
            maps_url = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
            results.append(f"[{name}]({maps_url}) - {address}")
    return results

# === Function: Get City Image from Unsplash ===
def get_city_image(city_name, access_key):
    url = f"https://api.unsplash.com/search/photos?query={city_name}&client_id={access_key}&per_page=1"
    response = requests.get(url)
    data = response.json()
    return data['results'][0]['urls']['regular'] if response.status_code == 200 and data['results'] else None

# === Function: Save Search to SQLite ===
def save_to_db(city, local_time, description, temp, feels_like):
    conn = sqlite3.connect("travel.db")
    c = conn.cursor()
    c.execute("""
        CREATE TABLE IF NOT EXISTS travel_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            city TEXT, local_time TEXT, description TEXT, temp REAL, feels_like REAL
        )
    """)
    c.execute("INSERT INTO travel_history (city, local_time, description, temp, feels_like) VALUES (?, ?, ?, ?, ?)",
              (city, local_time, description, temp, feels_like))
    conn.commit()
    conn.close()

# === Function: Show History ===
def show_history():
    conn = sqlite3.connect("travel.db")
    c = conn.cursor()
    c.execute("SELECT city, local_time, description, temp, feels_like FROM travel_history")
    rows = c.fetchall()
    conn.close()
    if rows:
        display(Markdown("## 📜 Past Travel Searches"))
        for row in rows:
            display(Markdown(f"**{row[0]}** at {row[1]} — {row[2]}, {row[3]}°C (Feels like {row[4]}°C)"))
    else:
        print("No history available.")


/Users/kellymao93/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ✅ 1. input city
city = input("Enter a city name: ")  

# ✅ 2. weather information
weather_info = get_weather(city)

# ✅ 3. show information
if "error" in weather_info:
    print("Error:", weather_info["error"])
else:
    display(Markdown(f"# 🌆 Travel Information for **{city}**"))

    # ✅ city image
    image_url = get_city_image(city, unsplash_key)
    if image_url:
        display(Image(url=image_url))

    # ✅ show weather information
    print(f"Local Time: {weather_info['local_time']}")
    print(f"Weather: {weather_info['description']}")
    print(f"Temperature: {weather_info['temperature']}°C (Feels like {weather_info['feels_like']}°C)")
    

    # ✅ store to database
    save_to_db(city, weather_info['local_time'], weather_info['description'],
               weather_info['temperature'], weather_info['feels_like'])

    # ✅ show exchange rate
    code = get_currency_code(weather_info["country"])
    if code:
        rate = get_exchange_rate("CAD", code)
        print(f"Currency Exchange: 1 CAD = {rate} {code}")

    # ✅ top 5 hotels, restaurants, attractions
    display(Markdown("## 🏨 Top 5 Hotels"))
    for i, hotel in enumerate(get_top_places(weather_info["lat"], weather_info["lon"], "lodging"), 1):
        display(Markdown(f"{i}. {hotel}"))

    display(Markdown("## 🍽️ Top 5 Restaurants"))
    for i, rest in enumerate(get_top_places(weather_info["lat"], weather_info["lon"], "restaurant"), 1):
        display(Markdown(f"{i}. {rest}"))

    display(Markdown("## 📍 Top 5 Attractions"))
    for i, att in enumerate(get_top_places(weather_info["lat"], weather_info["lon"], "tourist_attraction"), 1):
        display(Markdown(f"{i}. {att}"))
    view = input("Do you want to view your past travel queries? (yes/no): ")
    if view.lower() == "yes":
        show_history()



# 🌆 Travel Information for **Paris**

Local Time: 2025-06-22 09:05:31
Weather: clear sky
Temperature: 22.95°C (Feels like 22.84°C)
Currency Exchange: 1 CAD = 0.6327 EUR


## 🏨 Top 5 Hotels

1. [Britannique Hotel - Paris Centre](https://www.google.com/maps/place/?q=place_id:ChIJ-4Siox9u5kcRm9vM6DcMXyU) - 20 Avenue Victoria, Paris

2. [Hôtel Duo](https://www.google.com/maps/place/?q=place_id:ChIJWU9CqB1u5kcR5P1F2SMoJmc) - 11 Rue du Temple, Paris

...

## 📜 Past Travel Searches

**Tokyo** at 2025-06-22 12:17:40 — few clouds, 32.2°C (Feels like 34.73°C)

**Toronto** at 2025-06-21 23:26:24 — few clouds, 24.62°C (Feels like 24.94°C)

**Shanghai** at 2025-06-22 11:27:18 — broken clouds, 22.92°C (Feels like 23.57°C)

**Paris** at 2025-06-22 09:05:31 — clear sky, 22.95°C (Feels like 22.84°C)